In [96]:
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np
import os

#NEVER USE NP.ADD always use CV2.BITWISE_OR

In [97]:
imagePath = 'Images'
# Image path
imageName = "sample3.PNG"
imagePath = os.path.join(imagePath,imageName)
#road detection works better with images where the scale is smaller per unit pixel 
#(screen distance = lesser real world distance)
#It also works better with a different style of image

theta = (66.75*math.pi)/180    #diagonal angle FOV of camera (GIVEN!!)
phi = 2*math.atan(0.8*math.tan(theta/2))  #angle of view larger side of camera resolution (4 in 4:3)
omega = 2*math.atan(0.6*math.tan(theta/2))     #angle of view larger side of camera resolution (3 in 4:3)
alpha = (75*math.pi)/180   #set later on in the code based on the height of the camera [angle of camera from negative z axis]

# read image 
img = cv2.imread(imagePath)

mountingPoints = []
scalePoints = []
quadPoints = [[-1,-1],[-1,-1],[-1,-1]]

Steps to do test/implement ray masking
1. Plot out viewing quadrilateral
2. Plot out mounting point
3. Mask out for building contours
4. Find corners of building contours

Taking the nearest corner, find the slope of the line joining the mounting point and this nearest corner. Find the intersection point of this line and the farthest edge of the quadrilateral (What if farthest edge is out of bounds ??)

In [98]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [99]:
def getBorderContour_text(img):
#function runs only once
#using new bgr values for the new image in low red.

    img_c = img.copy()
    # Upper and lower color limit customized for snazzy maps
    low_red = (55, 55, 255)
    high_yellow = (242,251,256)

    low_gray = (241,241,241)
    high_gray = (244,243,241)

    # create masks
    red_mask = cv2.inRange(img, low_red, low_red )
    
    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    blank = np.zeros(img.shape, dtype='uint8')

    masked = cv2.bitwise_and(img,img,mask=combined_mask)

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
        area = cv2.contourArea(c)
        if(area>200):
            for eps in np.linspace(0.001, 0.01, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(blank, [approx], -1, (255,255,255), thickness=1)
            # cv2.drawContours(blank, [c], -1, 255, thickness=1)


    # cv2.imshow("image",img)
    # cv2.waitKey(0)

    return blank, masked

In [100]:
def click_MountingnScale_points(event, x, y, flags, params):

    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        mountingPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX


In [101]:
def getCorners(buildingBorderMask, quadPts, blocksize=3):

    # ~~~~~~~~~~~~~ #
    # for obtaining corners for ray masking
    # ~~~~~~~~~~~~~ #

    # Mask may have 3 layers
    operatedImage = cv2.cvtColor(buildingBorderMask, cv2.COLOR_BGR2GRAY)

    # uncomment if you feel image is noisy (not needed)
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) 

    # modify the data type -- setting to 32-bit floating point
    operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method to detect the corners with appropriate values as input parameters
    # increase the second parameter ~ blocksize to get more of the corner shape out
    dest = cv2.cornerHarris(operatedImage, blocksize, 5, 0.07)  

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # a mask of all corners of the building contours
    cornerMask = np.zeros(buildingBorderMask.shape, dtype='uint8')
    cornerMask[dest > 0.01 * dest.max()]=[255, 255, 255]

    # Make a gray scale mask of quadilateral area
    quadMask = np.zeros(buildingBorderMask.shape[:2], dtype='uint8')
    polyPts = np.array( quadPts ,dtype=np.int32)
    polyPts = polyPts.reshape((-1, 1, 2))
    quadMask = cv2.fillPoly(quadMask, pts=[polyPts],color=255)

    # a mask of all corners in quadilateral only
    cornerMask = cv2.bitwise_and(cornerMask,cornerMask,mask=quadMask)

    return cornerMask


In [102]:
def findClusterCenters(img, excludedCornerPoints = []):

    corner_centers = []

    onePixCornerMask = np.zeros(img.shape[:2], dtype='uint8')
    # blank_copy = blank.copy()

    white_mask = np.zeros(img.shape, dtype = 'uint8')

    if(len(img.shape)==3):
        white_mask = cv2.inRange(img, (255,255,255), (255,255,255))
    else:
        white_mask = cv2.inRange(img, 255, 255)


    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(white_mask, cv2.MORPH_DILATE,kernel)


    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for i in cnts:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
        # print(f"center - {cx},{cy}")
        if([cx,cy] not in excludedCornerPoints):
            onePixCornerMask[cy,cx] = 255
            # cv2.circle(onePixCornerMask, (cx, cy), 1, (255, 255, 255), -1)
            corner_centers.append([cx,cy])


    return corner_centers, onePixCornerMask


In [103]:
def getCorners_w_exclude(img, cornerPoints=[]):
    
    operatedImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # modify the data type
    # setting to 32-bit floating point
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
    # operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method
    # to detect the corners with appropriate
    # values as input parameters
    #~~~~~~~~~~~~~#
    #for obtaining corners for ray masking
    #~~~~~~~~~~~~~#
    dest = cv2.cornerHarris(operatedImage, 2, 5, 0.07)

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # for pts in poly_pts:
    #     dest[pts[0]][pts[1]] = 0
    # Reverting back to the original image,
    # with optimal threshold value
    size = img.shape

    cornerMask = np.zeros(size, dtype='uint8')
    cornerMask[dest > 0.01 * dest.max()]=[255, 255, 255]

    points,cornerMask = findClusterCenters(cornerMask,cornerPoints)

    return cornerMask, points

In [104]:
def pointGen(source, m, l):
    # m is the slope of line, and the
    # required Point lies distance l
    # away from the source Point
    a = Point(0, 0)
    b = Point(0, 0)

    # slope is 0
    if m == 0:
        a.x = source.x + l
        a.y = source.y

        b.x = source.x - l
        b.y = source.y

    # if slope is infinite
    elif math.isfinite(m) is False:
        a.x = source.x
        a.y = source.y + l

        b.x = source.x
        b.y = source.y - l
    else:
        dx = (l / math.sqrt(1 + (m * m)))
        dy = m * dx
        a.x = source.x + dx
        a.y = source.y + dy
        b.x = source.x - dx
        b.y = source.y - dy
    
    return [(int(a.x),int(a.y)),(int(b.x),int(b.y))]

In [105]:
def drawLine(slope, point, img):
    #draws line segment infinitely from edges of image

    returnImage = np.zeros(img.shape[:2], dtype = 'uint8')

    y_lim,x_lim = returnImage.shape
    px,py,qx,qz = [-1,-1,-1,-1] #initialize variables with 

    if(slope==-1):
        px = point[0]
        py = y_lim

        qx = point[0]
        qy = 0
    else:
        px = x_lim
        py = slope*x_lim + (point[1] - slope*point[0])

        qx = 0
        qy = (point[1] - slope*point[0])
        
    cv2.line(returnImage, (int(px),int(py)),(int(qx), int(qy)), 255, 1)
    return returnImage  



##### Step 1: Detect all corners in the image and mask out the view quadrilateral.. to get point only in the interior of the quad\n
##### Step 2: Mask out the view quadrilateral in the original img and then run corner detection.. to get all points interior and on edge
##### Step 3: Loop through every line of every contour inside quad
##### Step 4: For each line AND it with the interior corner points
##### Step 5: Isolate the points and their coordinates that intersect with the contour line
##### Step 6: Get the slope from the mounting point to those interior corners
##### Step 7: Drop a line from the interior corners to edge of image on a quad mask so you dont have to find intersection point with further edge
##### Step 8: Fill the quadrilateral made by the line contour, the lines from interior corners or the view quad side wall and the view quad further wall
##### Step 9: OR all the quadrilateral generated inside the loop on the view quad mask

In [106]:
## COLLECT MOUNTING POINTS

# read and display image 
img = cv2.imread(imagePath)
cv2.imshow("img",img)

# right click = view quadilateral (4+)
# left click = mounting point (1)
cv2.setMouseCallback('img', click_MountingnScale_points)
# points are to be selected in the order - 
# click 1 = nearest edge left point = quadPoints[0]
# click 2 = nearest edge right point = quadPoints[1]
# click 3 = further edge right point = quadPoints[2]
# click 4 = further edge left point = quadPoints[3]


cv2.waitKey(0)
cv2.destroyAllWindows()

In [107]:
scaleConst = 20 # for the below pixel distance, this represent
scalePoints = [(302, 373), (464, 373)]
# Get mounting points from building borders
# mountingPoints = get_MP_FromBuildings_ButAlsoText(img)

# Pixel distance of scale in image
# actual distance(m) = (scale constant)*(obtained magnitude)/scale
scale = abs(math.sqrt(pow(scalePoints[0][0] - scalePoints[1][0],2) + pow(scalePoints[0][1] - scalePoints[1][1],2)))

# heights = [x*0.1 for x in range(30,60)] #average height of light poles is 9 to 14 feet ~ 4.2m max
heights = [h*0.1 for h in range(30, 60)]    #3m to 6m
# Converting height in meter array to height in pixel array
heightPix = [h*scale/scaleConst for h in heights]

In [108]:
# angle of camera from negative of vertical axisS
img = cv2.imread(imagePath)
alpha = 60
ALPHA = alpha*math.pi/180

for mountingPoint in mountingPoints:
    for beta in range (0,30, 10):

        # print(beta)
        # beta edge cases
        if(beta==180 or beta==0 or beta==360):
            continue

        # loop vars
        BETA = beta*math.pi/180
        cameraRoadCoverage = np.zeros(img.shape[:2],dtype = 'uint8')

        # distances of closer and further edges from mounting point
        closer_dist = heightPix[20]*math.tan(ALPHA - (phi/2))
        further_dist = heightPix[20]*math.tan(ALPHA + (phi/2))

        # slope of horizontal plane camera angle
        slope_beta = math.tan(BETA)

        # midpoints of closer and further edges
        if beta > 180:
            closer_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, closer_dist)[1]
            further_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, further_dist)[1]
        else:
            closer_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, closer_dist)[0]
            further_midPoint = pointGen(Point(mountingPoint[0],mountingPoint[1]), slope_beta, further_dist)[0]

        # half of the closer edge length = (heightPix[20]*tan(w/2))/cos(alpha-(phi/2))
        # closer_edge = (heightPix[20]*math.tan(omega/2))/math.cos(ALPHA-(phi/2))
        # half of the further edge length = (heightPix[20]*tan(w/2))/cos(alpha+(phi/2))
        further_edge =(heightPix[20]*math.tan(omega/2))/math.cos(ALPHA+(phi/2))

        # Obtaining on ground quadilateral points
        # quadPoints[0] = pointGen(Point(closer_midPoint[0],closer_midPoint[1]), -1/slope_beta, closer_edge)[0]
        # quadPoints[1] = pointGen(Point(closer_midPoint[0],closer_midPoint[1]), -1/slope_beta, closer_edge)[1]

        quadPoints[0] = mountingPoint

        quadPoints[1] = pointGen(Point(further_midPoint[0],further_midPoint[1]), -1/slope_beta, further_edge)[1]
        quadPoints[2] = pointGen(Point(further_midPoint[0],further_midPoint[1]), -1/slope_beta, further_edge)[0]

# plotting the points
pt = np.array(quadPoints, np.int32)
pt = pt.reshape((-1,1,2))
cameraRoadCoverage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.polylines(cameraRoadCoverage, [pt],True, 255, 1)

cutQuadMask = np.zeros(img.shape[:2], dtype='uint8')
cutQuadBorder = np.zeros(img.shape[:2], dtype='uint8')
cv2.fillPoly(cutQuadMask, [pt], 255)

# findcontours
cnts=cv2.findContours(cutQuadMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
    area = cv2.contourArea(c)
    for eps in np.linspace(0.001, 0.01, 10):
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, eps * peri, True)
    
    # draw the approximated contour on the image  
    cv2.drawContours(cutQuadBorder, [approx], -1, (255,255,255), thickness=1)

print(pt)

cv2.imshow("area covered", cutQuadBorder)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[  17   23]]

 [[ 860  770]]

 [[1143   -8]]]


In [109]:
# h, w = img.shape[:2]
# imgBrdrPts = np.array([(0,0), (w, 0), (w,h), (0,h)], np.int32)
# imgOutlineMask = np.zeros(img.shape[:2], dtype="uint8")
# imgBrdrPts = imgBrdrPts.reshape((-1,1,2))
# cv2.polylines(imgOutlineMask, [imgBrdrPts],True, 255, 5)
# imgOutlineMask = cv2.bitwise_not(imgOutlineMask)

In [110]:
quadMask = cutQuadMask
bldg_brdrs, _ = getBorderContour_text(img)

# get building borders inside viewing quadrilateral
selected_bldg_brdrs = cv2.bitwise_and(bldg_brdrs,bldg_brdrs, mask = quadMask)
selected_bldg_brdrs_gray = cv2.cvtColor(selected_bldg_brdrs, cv2.COLOR_BGR2GRAY) # Gray

# 
selected_edge_list = cv2.findContours(selected_bldg_brdrs_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
selected_edge_list = selected_edge_list[0] if len(selected_edge_list) == 2 else selected_edge_list[1]

# all_selected_corner = cv2.bitwise_and(bldg_brdrs, quadMask)
# all_selected_corner = getCorners(all_selected_corner)

# building corners inside viewing quadrilateral
original_corners = getCorners(bldg_brdrs, quadPoints)
original_corners_g = cv2.cvtColor(original_corners, cv2.COLOR_BGR2GRAY)

In [111]:
# Step 1: Detect all corners in the image and mask out the view quadrilateral.. to get point only in the interior of the quad

_, cornerPix = findClusterCenters(original_corners) #cornerPix is all corner on the inside of the view quadrilateral

# Points displayed on img
# imgCornerPix = np.add(img, original_corners)

cv2.imshow("bldg_brdrs",selected_bldg_brdrs)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [112]:
# Step 2: Mask out the view quadrilateral in the original img and then run corner detection.. to get all points interior and on edge

# bldg_borders_view_masked = cv2.bitwise_and(bldg_brdrs, bldg_brdrs, mask = quadMask)
# all_corners_found = getCorners(bldg_borders_view_masked, quadPoints)
# all_corners_found = findClusterCenters(all_corners_found)   #all corners on the interior and on edge

# blank = np.zeros(img.shape[:2], dtype='uint8')
# nearestEdgeMask = cv2.line(blank, quadPoints[0], quadPoints[1], 255, 1)   #plot nearest edge

# blank = np.zeros(img.shape[:2], dtype='uint8')
# farthestEdgeMask = cv2.line(blank, quadPoints[2], quadPoints[3], 255, 1)   #plot farthest edge

# # cv2.imshow("farthest edge", farthestEdgeMask)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()

# quad_outline = np.zeros(bldg_brdrs.shape[:2], dtype='uint8')
# cv2.polylines(quad_outline, [pt],True, 255, 1)
# mountingPoint = mountingPoints[0]

In [113]:
# Step 3: Loop through every line of every contour inside quad
# for selected_edge_contour in selected_edge_list:

selected_edge_contour = selected_edge_list[0]

#mask related to a single contour, to which individual line blocking masks will be OR'ed to 
totalMask = np.zeros(img.shape[:2], dtype = 'uint8')

oneContourPic = np.zeros(img.shape, dtype='uint8')
cv2.drawContours(oneContourPic, [selected_edge_contour], -1, (255,255,255), thickness=1) #draws only selected contour for the loop
oneContourPic = cv2.cvtColor(oneContourPic, cv2.COLOR_BGR2GRAY)   #convert drawn picture into grayscale

cv2.imshow("one contour", oneContourPic)
cv2.waitKey(0)
cv2.destroyAllWindows()

oneContourPic = cv2.Canny(oneContourPic,50,150,apertureSize=3)
# cv2.imshow("one Contour", oneContourPic)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Apply HoughLinesP method to 
# to directly obtain line end points
lines = cv2.HoughLinesP(
            oneContourPic, # Input edge image
            1, # Distance resolution in pixels
            np.pi/180, # Angle resolution in radians
            threshold=10, # Min number of votes for valid line. Use lesser number of votes for smaller lines
            minLineLength=5, # Min allowed length of line
            maxLineGap=10 # Max allowed gap between line for joining them
            )
for points in lines:

    # Extracted points nested in the list
    x1,y1,x2,y2=points[0]

    # Step 4: For each line AND it with the interior corner points ??

    interiorPointCoords_1 = [y1, x1]
    interiorPointCoords_2 = [y2, x2]

    # Step 5: Isolate the points and their coordinates that intersect with the contour line
    # Step 6: Get the slope from the mounting point to those interior corners
    # Step 7: Drop a line from the interior corners to edge of image on a quad mask so you dont have to find intersection point with further edge
    
    joiningLine_1 = drawLine((mountingPoint[1] - interiorPointCoords_1[1])/(mountingPoint[0] - interiorPointCoords_1[0]), interiorPointCoords_1, img)
    joiningLine_2 = drawLine((mountingPoint[1] - interiorPointCoords_2[1])/(mountingPoint[0] - interiorPointCoords_2[0]), interiorPointCoords_2, img)
    
    Point_1 = cv2.bitwise_and(cutQuadBorder, joiningLine_1)
    PointCoords_1, t1 = findClusterCenters(Point_1)  #more than one cluster might form or no cluster might wrong

    Point_2 = cv2.bitwise_and(cutQuadBorder, joiningLine_2)
    PointCoords_2, _ = findClusterCenters(Point_2)  #more than one cluster might form or no cluster might wrong
    # farthestEdgePointCoords_1 = [-1,-1]
    # farthestEdgePointCoords_2 = [-1,-1]

    if(len(Point_1)==0 and len(Point_2)==0):    #i.e. points are not found
        PointCoords_1 = PointCoords_1[0]
        PointCoords_2 = PointCoords_2[0]

# cv2.imshow("Farthest Edge Point", Point_2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

    print(PointCoords_1, PointCoords_2, interiorPointCoords_1, interiorPointCoords_2)

    lineMask = np.zeros(img.shape[:2], dtype='uint8')

    # Step 8: Fill the quadrilateral made by the line contour, the lines from interior corners or the view quad side wall and the view quad further wall
    # There is a concern that the points are listed in not the correct order...
    blockPts = np.array( [interiorPointCoords_1, interiorPointCoords_2, PointCoords_1[0], PointCoords_2[0]], dtype=np.int32)
    blockPts = blockPts.reshape((-1, 1, 2))
    lineMask = cv2.fillPoly(lineMask, pts=[blockPts],color=255)

    # Step 9: OR all the quadrilateral generated inside the loop on the view quad mask
    totalMask = cv2.bitwise_or(totalMask, lineMask)


totalMask = cv2.bitwise_not(totalMask)

cv2.imshow("Farthest Edge Point", totalMask)
cv2.waitKey(0)
cv2.destroyAllWindows()



[[19, 25]] [[17, 23]] [372, 410] [300, 413]
[] [] [301, 471] [301, 486]


IndexError: list index out of range